In [188]:
import pandas as pd
import xlwt
import xlrd
import openpyxl

### xlrd

In [5]:
"""xlsx文件内容如下,s1,s2两个sheet，s2内容随机"""
s1 = pd.DataFrame([['wanghl',12,True,'','2020/1/2'],range(3),list('abcd')],columns=list('ABCDE'),index=range(1,4))
s1

,A,B,C,D,E
1,wanghl,12,True,,2020/1/2
2,0,1,2,None,None
3,a,b,c,d,None


In [6]:
"""打开xlsx"""
book = xlrd.open_workbook('test.xlsx')

In [175]:
"""获取sheet names列表"""
print(book.sheet_names())
"""判断sheet是否导入成功，也就是sheet_names中的不一定导入成功，按sheet位置或名称判断都可以"""
book.sheet_loaded(0),book.sheet_loaded('s1')

['s1', 's2']


(True, True)

In [176]:
"""会以最右下角数据为边界加载整个表格。例：如果仅有1个数据但是在整个sheet的右下角，也会加载最大的sheet表格"""
"""加载所有sheet"""
shee = book.sheets()
print(shee)
"""加载个别sheet"""
she1 = book.sheet_by_index(0)
she2 = book.sheet_by_name('s1')
print(she1)
print(she2)
she1 == she2

[<xlrd.sheet.Sheet object at 0x000001CFDD861B70>, <xlrd.sheet.Sheet object at 0x000001CFDD861E80>]


True

In [193]:
"""加载sheet，行操作"""
she = book.sheet_by_index(0)
"""统计sheet总行数"""
print(she.nrows)  # 获取总行数
print(she.row_len(1))  # 获取某一行长度
print(she.row(0))  # 获取一行数据,返回列表
"""以下三个函数(rowx[, start_colx=0, end_colx=None])"""
print(she.row_types(0))  # 获取cell类型：empty为0，string为1，number为2，date为3，boolean为4， error为5
print(she.row_slice(0,1,4))  # 获取某行数据，可切片
print(she.row_values(0,1,4))  # 只获取值返回列表，可切片

3
5
[text:'wanghl', number:12.0, bool:1, empty:'', xldate:43832.0]
<class 'xlrd.sheet.Cell'>
12.0
array('B', [1, 2, 4, 0, 3])
[number:12.0, bool:1, empty:'']
[12.0, 1, '']


In [197]:
"""返回行的生成器"""
ite = she.get_rows()
for i in range(3):
    print(next(ite))

[text:'wanghl', number:12.0, bool:1, empty:'', xldate:43832.0]
[number:1.0, number:2.0, number:3.0, empty:'', empty:'']
[text:'a', text:'b', text:'c', text:'d', empty:'']


In [10]:
"""加载sheet，列操作"""
she = book.sheet_by_index(0)
"""统计sheet总列数"""
print(she.ncols)  # 总列数
"""以下三个函数(colx[, start_rowx=0, end_rowx=None])"""
print(she.col_types(0))  # 获取cell类型：empty为0，string为1，number为2，date为3，boolean为4， error为5
print(she.col_slice(0,0,3))  # 获取某列数据，可切片
print(she.col_values(0,0,3))  # 只或取值返回列表，可切片

5
[1, 2, 1]
[text:'wanghl', number:1.0, text:'a']
['wanghl', 1.0, 'a']


In [32]:
"""加载sheet，单元格操作"""
she = book.sheet_by_index(0)
"""获取某单元格"""
print(she.cell(0,4))
print(she.cell_value(0,4))
print(she.cell_type(0,4))
print(she.cell(0,4).value)  # 获取cell中的值

"""xldata单元格转换为日期，book.datamode中的book是book = xlrd.open_workbook('test.xlsx')"""
print(xlrd.xldate_as_tuple(she.cell_value(0,4), book.datemode))

xldate:43832.0
43832.0
3
43832.0
(2020, 1, 2, 0, 0, 0)


In [187]:
"""若获取合并单元格信息，需设置formatting_info参数，xls格式设置为True，xlsx格式设置为默认值False"""
bk = xlrd.open_workbook('t.xls',formatting_info=True)
she = bk.sheet_by_index(0)
"""获取sheet中合并单元格的信息，返回值为列表。若sheet对象中无合并单元格，则返回值为空列表。
格式为(row_start, row_end, col_start, col_end)，范围不包括row_end,col_end。
"""
mgcell = she.merged_cells
print(mgcell[:3])
"""使用row_start和col_start就可以获取cell值"""
print(she.cell(1218,0))

[(1215, 1216, 17, 19), (1218, 1220, 0, 1), (1213, 1215, 0, 1)]
text:'脱碳' (XF:161)


### xlwt

In [202]:
import time

In [209]:
"""创建excel"""
wb = xlwt.Workbook(encoding='utf8')

In [210]:
"""创建sheet名称为s1，cell_overwrite_ok=True表示可重复写入同一个单元格"""
ws = wb.add_sheet(u's1',  cell_overwrite_ok=True)

In [211]:
"""写入cell，格式write(r_num,c_num,value,style)"""
ws.write(0, 0, time.ctime(),xlwt.easyxf(num_format_str='D-MMM-YY'))

In [ ]:
"""公式"""
ws.write(1, 0, xlwt.Formula('A3+B3'))
"""超链接"""
ws.write(2, 0, xlwt.Formula('HYPERLINK("https://www.baidu.com/";"bdu")'))
"""合并单元格(row_start, row_end, col_start, col_end)"""
ws.write_merge(22, 29, 5, 8, 'wanghl',style_x)
"""列宽调节,每256是一个字符"""
ws.col(0).width = 25600
"""可以通过自动换行通过输入文字的多少来确定行高"""
stytall_style = xlwt.easyxf('font:height 720;') # 36pt,类型小初的字号
ws.row(0).set_style(tall_style)
"""插入图片,只能插入bmp格式的。scale_x调节图片缩放比例。Row,col指定图片左上角位置。"""
ws.insert_bitmap(filename, row, col, x = 0, y = 0, scale_x = 1, scale_y = 1):  

In [213]:
"""保存"""
wb.save('tt.xls')

In [ ]:
"""style示例"""
style_0 = xlwt.easyxf('font: name Times New Roman, color-index red, bold on', num_format_str='#,##0.00')

In [ ]:
"""字体"""
style_1 = xlwt.XFStyle() # 初始化样式
font = xlwt.Font() # 为样式创建字体
font.name = 'Times New Roman' 
font.bold = True # 黑体
font.colour_index = 7
font.underline = True # 下划线
font.italic = True # 斜体字
style_1.font = font # 设定样式
## font其他属性
font.struck_out = True
font.family = xlwt.Font.FAMILY_ROMAN
font.charset = xlwt.Font.CHARSET_ANSI_LATIN
font.get_biff_record = 
font.height = 0x00c8
font.outline =
font.shadow =

In [ ]:
"""日期"""
style_3 = xlwt.XFStyle()
style_3.num_format_str = 'M/D/YY'

In [ ]:
"""对齐
HORZ_GENERAL, HORZ_LEFT, HORZ_CENTER, HORZ_RIGHT, HORZ_FILLED, HORZ_JUSTIFIED, HORZ_CENTER_ACROSS_SEL, 
HORZ_DISTRIBUTED, VERT_TOP, VERT_CENTER, VERT_BOTTOM, VERT_JUSTIFIED, VERT_DISTRIBUTED"""
style_4 = xlwt.XFStyle() # Create Style
alignment = xlwt.Alignment() # Create Alignment
alignment.horz = xlwt.Alignment.HORZ_CENTER 
alignment.vert = xlwt.Alignment.VERT_CENTER
style_4.alignment = alignment # Add Alignment to Style

In [ ]:
"""边框
DASHED虚线，NO_LINE没有，THIN实线，
NO_LINE, THIN, MEDIUM, DASHED, DOTTED, THICK, DOUBLE, HAIR, MEDIUM_DASHED, THIN_DASH_DOTTED, 
MEDIUM_DASH_DOTTED, THIN_DASH_DOT_DOTTED, MEDIUM_DASH_DOT_DOTTED, SLANTED_MEDIUM_DASH_DOTTED, 
0x00 through 0x0D"""
style_5 = xlwt.XFStyle() # Create Style
borders = xlwt.Borders() # Create Borders
borders.left = xlwt.Borders.DASHED 
borders.right = xlwt.Borders.DASHED
borders.top = xlwt.Borders.DASHED
borders.bottom = xlwt.Borders.DASHED
borders.left_colour = 0x40
borders.right_colour = 0x40
borders.top_colour = 0x40
borders.bottom_colour = 0x40
style_5.borders = borders # Add Borders to Style

In [ ]:
"""背景色"""
"""NO_PATTERN, SOLID_PATTERN, or 0x00 through 0x12,
8 through 63. 0 = Black, 1 = White, 2 = Red, 3 = Green, 4 = Blue, 5 = Yellow, 6 = Magenta, 
7 = Cyan, 16 = Maroon, 17 = Dark Green, 18 = Dark Blue, 19 = Dark Yellow , almost brown, 
20 = Dark Magenta, 21 = Teal, 22 = Light Gray, 23 = Dark Gray, the list goes on..."""
"""pattern.pattern_fore_colour = 0x00黑色,0x01白,0x02红,0x03浅绿,0x04蓝,0x05黄,0x06粉,0x07青,0x18深蓝"""
style_6 = xlwt.XFStyle() # Create Style
pattern = xlwt.Pattern() # Create the Pattern
pattern.pattern = xlwt.Pattern.SOLID_PATTERN 
pattern.pattern_fore_colour = 5 
pattern.pattern_back_colour = 5 
style_6.pattern = pattern # Add Pattern to Style

### openpyxl